# Kotlin NDArrays

Thanks to **Multik** and the standard Collections library, it's easy to create, access and manipulate N-dimensional arrays and matrices.

You can import Multik using Jupyter's magic command `%use`, for importing Multik library and dependencies.

In [1]:
%use multik

We can create a simple 1D array as follows (type declaration is not mandatory)

In [3]:
val arr: NDArray<Double, D1> = mk.arange(10)
arr

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]